# Llama Impact Hackathon (Part 2 - Vision)

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [217]:
# Read the config file for Together AI API key
import json
with open('configs.json', 'r') as file:
    config = json.load(file)

print('Configs Read!')

Configs Read!


In [98]:
# Read csv file
sample_df = pd.read_csv('sample_df-1-translation.csv')
sample_df.head()

,web-scraper-order,web-scraper-start-url,complaint,complaint-href,title,user_text,images-src,municipality,complaint_number,image_url,complaint_en,title_en,user_text_en,date_created,date_modified
0,1731179571-1,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 18:50, 7 Mar 2017, ...",https://ndreqe.com/report/171,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga ne lokacionin e shenuar nuk eshte shtru...,/photo/171.0.jpeg?876fef51,Prishtinës,171,https://ndreqe.com/photo/171.0.full.jpeg?876fef51,"Public Domain 18:50, 7 Mar 2017, Last Update 1...",Reported in Public Illumination category by an...,The street with the designated location is not...,2017-03-07 18:50:00,2017-03-09 16:00:00
1,1731179574-2,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.0.jpeg?eefb909c,Prishtinës,189,https://ndreqe.com/photo/189.0.full.jpeg?eefb909c,"Public domain 17:09, 10 Mar 2017, Last update ...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00
2,1731179574-3,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.1.jpeg?dd7ac08d,Prishtinës,189,https://ndreqe.com/photo/189.1.jpeg?dd7ac08d,"Public release 17:09, 10 Mar 2017, Last update...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00
3,1731179577-4,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.0.jpeg?e7281615,Prishtinës,365,https://ndreqe.com/photo/365.0.full.jpeg?e7281615,"Page protected 04:57, 2 Apr 2017, Last updated...",Reported in Public Lighting category by anonym...,Ferid Curri Street does not have public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00
4,1731179577-5,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.1.jpeg?6898da66,Prishtinës,365,https://ndreqe.com/photo/365.1.jpeg?6898da66,"Public domain 04:57, 2 Apr 2017, Last updated ...",Reported in the Public lighting category by an...,Ferid Curri Street has no public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00


# Vision Operations

In [103]:
!pip install together

In [ ]:
# Set up the LLM Client
from together import Together

client = Together(api_key=config['api_key'])
visionModel = "meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo"

In [106]:
# old prompt
imagePrompt = """
You are an agent who verbally describes an image. Describe the provided image in detail. The image would be a photo of a municipality complaint in Kosovo, provided by the public on their municipality complaints portal.
I will feed in the output you give me to a severity assessment model that will quantify the severity of the problem, so please think step by step and describe the problem in detail.
Pay close attention to things in the image, and see how would this disrupt a resident in the area.
And here is some additional description from the person who submitted this photo:
"""

In [107]:
complaint_row = 1
imageUrl = sample_df['image_url'].iloc[complaint_row]
complaintDetails = sample_df['title_en'].iloc[complaint_row] + sample_df['user_text_en'].iloc[complaint_row]


response = client.chat.completions.create(
    model=visionModel,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": imagePrompt+complaintDetails},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": imageUrl,
                    },
                },
            ],
        }
    ]
)
response.choices[0].message.content

'The image shows a field with a utility pole in the center, surrounded by bare trees and bushes. In the background, there are several houses that appear to be residential.\n\nThe utility pole is the main focus of the image, as it is the subject of the complaint. The pole is tall and slender, with a flat top and a metal crossbar at the base. It appears to be made of wood or metal, but the exact material is not clear.\n\nIn the foreground, the field is dry and brown, with no signs of vegetation or life. The ground is covered in a layer of dead grass and weeds, and there are no visible paths or roads. The sky above is overcast and gray, adding to the sense of desolation and neglect.\n\nOverall, the image suggests that the utility pole has been installed for some time, but it is not functioning properly. The fact that the light does not work since it was installed implies that there may be a technical issue or a problem with the installation process. The surrounding environment, which appe

In [120]:
# Make a function out of it

def get_image_description(row):
  imageUrl = row['image_url']
  complaintDetails = row['title_en'] + '\n\n' + row['user_text_en']

  response = client.chat.completions.create(
      model=visionModel,
      messages=[
          {
              "role": "user",
              "content": [
                  {"type": "text", "text": imagePrompt+complaintDetails+'\n\n'},
                  {
                      "type": "image_url",
                      "image_url": {
                          "url": imageUrl,
                      },
                  },
              ],
          }
      ],
      temperature=0.5
  )
  time.sleep(0.5)
  return response.choices[0].message.content


print('Image Description Function defined!')

Image Description Function defined!


In [121]:
# Test the code
sample_df.iloc[:1][['image_url','title_en','user_text_en']].apply(get_image_description, axis=1).iloc[0]

"The image shows a map of a municipality in Kosovo, with a red line indicating a road that is not paved and lacks public lighting. The road is located in a rural area, surrounded by fields and houses. The surrounding area appears to be a residential neighborhood with several houses and buildings, including a school or community center.\n\nThe image also includes several yellow arrows and text labels, which appear to be in Albanian. These labels likely provide additional information about the issue, such as the location of the problem and the status of any requests for repairs or maintenance.\n\nOverall, the image suggests that the municipality has received a complaint about the poor condition of the road and the lack of public lighting in the area. The complaint has been reported to the commune, but no response has been received yet.\n\nThe image provides a clear visual representation of the problem, which is essential for assessing its severity. The fact that the road is not paved and

In [122]:
# Apply the code on the sample_df
sample_df['image_description'] = sample_df[['image_url','title_en','user_text_en']].apply(get_image_description, axis=1)
sample_df.head()

,web-scraper-order,web-scraper-start-url,complaint,complaint-href,title,user_text,images-src,municipality,complaint_number,image_url,complaint_en,title_en,user_text_en,date_created,date_modified,image_description
0,1731179571-1,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 18:50, 7 Mar 2017, ...",https://ndreqe.com/report/171,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga ne lokacionin e shenuar nuk eshte shtru...,/photo/171.0.jpeg?876fef51,Prishtinës,171,https://ndreqe.com/photo/171.0.full.jpeg?876fef51,"Public Domain 18:50, 7 Mar 2017, Last Update 1...",Reported in Public Illumination category by an...,The street with the designated location is not...,2017-03-07 18:50:00,2017-03-09 16:00:00,The image shows a map of a municipality in Kos...
1,1731179574-2,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.0.jpeg?eefb909c,Prishtinës,189,https://ndreqe.com/photo/189.0.full.jpeg?eefb909c,"Public domain 17:09, 10 Mar 2017, Last update ...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00,The image shows a utility pole with a solar-po...
2,1731179574-3,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.1.jpeg?dd7ac08d,Prishtinës,189,https://ndreqe.com/photo/189.1.jpeg?dd7ac08d,"Public release 17:09, 10 Mar 2017, Last update...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00,The image shows a streetlight with a broken li...
3,1731179577-4,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.0.jpeg?e7281615,Prishtinës,365,https://ndreqe.com/photo/365.0.full.jpeg?e7281615,"Page protected 04:57, 2 Apr 2017, Last updated...",Reported in Public Lighting category by anonym...,Ferid Curri Street does not have public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00,The image shows a dark street at night with no...
4,1731179577-5,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.1.jpeg?6898da66,Prishtinës,365,https://ndreqe.com/photo/365.1.jpeg?6898da66,"Public domain 04:57, 2 Apr 2017, Last updated ...",Reported in the Public lighting category by an...,Ferid Curri Street has no public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00,The image depicts a nighttime scene of a road ...


In [123]:
# Save file
sample_df.to_csv('sample_df-2-imageDescriptions.csv', index=False)
print('File saved successfully!')

File saved successfully!


# Severity Assessment

In [124]:
# Read the file
sample_df2 = pd.read_csv('sample_df-2-imageDescriptions.csv')
sample_df2.head()

,web-scraper-order,web-scraper-start-url,complaint,complaint-href,title,user_text,images-src,municipality,complaint_number,image_url,complaint_en,title_en,user_text_en,date_created,date_modified,image_description
0,1731179571-1,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 18:50, 7 Mar 2017, ...",https://ndreqe.com/report/171,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga ne lokacionin e shenuar nuk eshte shtru...,/photo/171.0.jpeg?876fef51,Prishtinës,171,https://ndreqe.com/photo/171.0.full.jpeg?876fef51,"Public Domain 18:50, 7 Mar 2017, Last Update 1...",Reported in Public Illumination category by an...,The street with the designated location is not...,2017-03-07 18:50:00,2017-03-09 16:00:00,The image shows a map of a municipality in Kos...
1,1731179574-2,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.0.jpeg?eefb909c,Prishtinës,189,https://ndreqe.com/photo/189.0.full.jpeg?eefb909c,"Public domain 17:09, 10 Mar 2017, Last update ...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00,The image shows a utility pole with a solar-po...
2,1731179574-3,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.1.jpeg?dd7ac08d,Prishtinës,189,https://ndreqe.com/photo/189.1.jpeg?dd7ac08d,"Public release 17:09, 10 Mar 2017, Last update...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00,The image shows a streetlight with a broken li...
3,1731179577-4,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.0.jpeg?e7281615,Prishtinës,365,https://ndreqe.com/photo/365.0.full.jpeg?e7281615,"Page protected 04:57, 2 Apr 2017, Last updated...",Reported in Public Lighting category by anonym...,Ferid Curri Street does not have public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00,The image shows a dark street at night with no...
4,1731179577-5,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.1.jpeg?6898da66,Prishtinës,365,https://ndreqe.com/photo/365.1.jpeg?6898da66,"Public domain 04:57, 2 Apr 2017, Last updated ...",Reported in the Public lighting category by an...,Ferid Curri Street has no public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00,The image depicts a nighttime scene of a road ...


In [ ]:
client = Together(api_key=config['api_key'])
severity_model = "meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo"

In [128]:
# Load the severity framework
severity_framework = pd.read_csv('severity-framework.csv', sep=';', encoding='cp1252')
severity_framework

,Level,Level Name,Description
0,0,Nothing,No issue identified. Example: Complaint lacks ...
1,1,Minimal,Very low-impact issues that cause no harm or d...
2,2,Very Low,Low-impact issues causing minor inconvenience ...
3,3,Low,Minor issues that may require eventual attenti...
4,4,Moderate,Issues affecting convenience or aesthetics tha...
5,5,Noticeable,Issues that start to impact public comfort or ...
6,6,Significant,Issues that affect the daily life of residents...
7,7,High,"Problems that impact public safety, comfort, o..."
8,8,Very High,Severe issues causing major disruption or pote...
9,9,Critical,"Critical problems with widespread impact, risk..."


In [129]:
# Create this dataframe into a string
# Choose your value separator (e.g., comma, tab, or other meaningful separator)
value_separator = ', '
row_separator = '\n'

# Convert dataframe to string
severity_framework_string = row_separator.join(value_separator.join(map(str, row)) for row in severity_framework.values)
severity_framework_string

"0, Nothing, No issue identified. Example: Complaint lacks validity, or issue is not within the municipality's responsibilities (e.g., a misunderstanding or non-actionable complaint).\n1, Minimal, Very low-impact issues that cause no harm or disruption. Example: Aesthetic complaints, like minor graffiti or slightly faded street signs that don’t affect readability or public safety.\n2, Very Low, Low-impact issues causing minor inconvenience to individuals but not affecting safety or the community at large. Example: Small amounts of litter on streets, slightly cracked pavement that doesn’t pose an immediate hazard.\n3, Low, Minor issues that may require eventual attention but aren’t urgent or disruptive. Example: Overgrown bushes or trees blocking signs, slightly uneven sidewalks in low-traffic areas.\n4, Moderate, Issues affecting convenience or aesthetics that need attention but don’t pose an immediate hazard. Example: Broken benches in public parks, moderate graffiti on public propert

Now let's try with the severity framework and see how well and consistently does the LLM assess the complaint's severity

In [180]:
severity_framework_system_prompt = """
Context:
We're dealing with municipality complaints from the public in Kosovo. Complaints are uploaded on a platform, and the municipal committee works on the complaints to resolve them.
Now we're developing an LLM based framework to assist the municipal management committee to identify problems with the limited knowledge, and to help prioritizing the management.

Task:
I will provide you a severity framework as a reference. And then, I will provide you descriptions and a picture of the complaint.
You will look at the complaint description and photo, and score the severity of the complaint based on the framework given to you.
Pay attention to the small details, and to the bigger picture, and base your severity on that.

Severity Framework:
"""+ severity_framework_string + """

Output:
Your output should be a number between 0 and 10. Only output the number in your response. Avoid any words. Avoid any punctuation.

The complaint details are: \n
"""


complaint_row = 0
image_url = sample_df['image_url'].iloc[complaint_row]
description = sample_df['title_en'].iloc[complaint_row] + sample_df['user_text_en'].iloc[complaint_row] + sample_df['image_description'].iloc[complaint_row]


response = client.chat.completions.create(
    model=severity_model,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": severity_framework_system_prompt+description},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": imageUrl,
                    },
                },
            ],
        }
    ]
    , temperature=0.1
)
response.choices[0].message.content

'6'

In [186]:
# Package this in a function

def get_complaint_severity(row):
  image_url = row['image_url']
  description = row['title_en'] + row['user_text_en'] + row['image_description']


  response = client.chat.completions.create(
      model=severity_model,
      messages=[
          {
              "role": "user",
              "content": [
                  {"type": "text", "text": severity_framework_system_prompt+description},
                  {
                      "type": "image_url",
                      "image_url": {
                          "url": imageUrl,
                      },
                  },
              ],
          }
      ]
      , temperature=0.1
  )
  time.sleep(0.3)
  return response.choices[0].message.content


print('Complaint severity Function defined!')

Complaint severity Function defined!


In [188]:
# Apply the code on the sample_df
sample_df['complaint_severity'] = sample_df[['image_url','title_en','user_text_en','image_description']].apply(get_complaint_severity, axis=1)
sample_df['complaint_severity'] = pd.to_numeric(sample_df['complaint_severity'], errors='coerce').astype('Int64')
sample_df.head()

,web-scraper-order,web-scraper-start-url,complaint,complaint-href,title,user_text,images-src,municipality,complaint_number,image_url,complaint_en,title_en,user_text_en,date_created,date_modified,image_description,complaint_severity
0,1731179571-1,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 18:50, 7 Mar 2017, ...",https://ndreqe.com/report/171,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga ne lokacionin e shenuar nuk eshte shtru...,/photo/171.0.jpeg?876fef51,Prishtinës,171,https://ndreqe.com/photo/171.0.full.jpeg?876fef51,"Public Domain 18:50, 7 Mar 2017, Last Update 1...",Reported in Public Illumination category by an...,The street with the designated location is not...,2017-03-07 18:50:00,2017-03-09 16:00:00,The image shows a map of a municipality in Kos...,6
1,1731179574-2,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.0.jpeg?eefb909c,Prishtinës,189,https://ndreqe.com/photo/189.0.full.jpeg?eefb909c,"Public domain 17:09, 10 Mar 2017, Last update ...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00,The image shows a utility pole with a solar-po...,7
2,1731179574-3,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.1.jpeg?dd7ac08d,Prishtinës,189,https://ndreqe.com/photo/189.1.jpeg?dd7ac08d,"Public release 17:09, 10 Mar 2017, Last update...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00,The image shows a streetlight with a broken li...,7
3,1731179577-4,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.0.jpeg?e7281615,Prishtinës,365,https://ndreqe.com/photo/365.0.full.jpeg?e7281615,"Page protected 04:57, 2 Apr 2017, Last updated...",Reported in Public Lighting category by anonym...,Ferid Curri Street does not have public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00,The image shows a dark street at night with no...,7
4,1731179577-5,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.1.jpeg?6898da66,Prishtinës,365,https://ndreqe.com/photo/365.1.jpeg?6898da66,"Public domain 04:57, 2 Apr 2017, Last updated ...",Reported in the Public lighting category by an...,Ferid Curri Street has no public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00,The image depicts a nighttime scene of a road ...,7


In [189]:
# Save file
sample_df.to_csv('sample_df-3-severity.csv', index=False)
print('File saved successfully!')

File saved successfully!


In [193]:
sample_df.complaint_severity.value_counts()

,count
complaint_severity,
6,38
7,25
8,18
5,12
3,6


# Create Categories

In [198]:
categories = 'Rubbish, Pothole in the Road, Other, Damaged Sidewalk, Environment, Public Lighting, Sewage, Road Sign'

In [205]:
category_system_prompt = """
I have a list of categories: \n
"""+ categories +"""\n\n

Input:
I will give you a text.

Task:
You will tell me which category does the text belong to.

Output:
Your output should only contain the category, and nothing else. Avoid any extra words. Avoid any punctuation.

The input text is: \n
"""


complaint_row = 50
complaint = sample_df['title_en'].iloc[complaint_row]


response = client.chat.completions.create(
    model=severity_model,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": category_system_prompt+complaint},
            ],
        }
    ]
    , temperature=0.1
)
response.choices[0].message.content

'Rubbish'

In [208]:
# Package it in a function
def get_category(title):
  response = client.chat.completions.create(
      model=severity_model,
      messages=[
          {
              "role": "user",
              "content": [
                  {"type": "text", "text": category_system_prompt+title},
              ],
          }
      ]
      , temperature=0.5
  )
  time.sleep(1)
  return response.choices[0].message.content

print('Function defined successfully!')

Function defined successfully!


In [210]:
# Apply the code on the sample_df
sample_df['category'] = sample_df['title_en'].apply(get_category)
sample_df.head()

,web-scraper-order,web-scraper-start-url,complaint,complaint-href,title,user_text,images-src,municipality,complaint_number,image_url,complaint_en,title_en,user_text_en,date_created,date_modified,image_description,complaint_severity,category
0,1731179571-1,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 18:50, 7 Mar 2017, ...",https://ndreqe.com/report/171,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga ne lokacionin e shenuar nuk eshte shtru...,/photo/171.0.jpeg?876fef51,Prishtinës,171,https://ndreqe.com/photo/171.0.full.jpeg?876fef51,"Public Domain 18:50, 7 Mar 2017, Last Update 1...",Reported in Public Illumination category by an...,The street with the designated location is not...,2017-03-07 18:50:00,2017-03-09 16:00:00,The image shows a map of a municipality in Kos...,6,Public Lighting
1,1731179574-2,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.0.jpeg?eefb909c,Prishtinës,189,https://ndreqe.com/photo/189.0.full.jpeg?eefb909c,"Public domain 17:09, 10 Mar 2017, Last update ...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00,The image shows a utility pole with a solar-po...,7,Public Lighting
2,1731179574-3,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 17:09, 10 Mar 2017, ...",https://ndreqe.com/report/189,Raportuar në kategorinë Ndriçim publik nga ano...,Drita nuk punon prej se eshte instaluar.,/photo/189.1.jpeg?dd7ac08d,Prishtinës,189,https://ndreqe.com/photo/189.1.jpeg?dd7ac08d,"Public release 17:09, 10 Mar 2017, Last update...",Reported in the Public Lighting category by an...,The light does not work since it was installed.,2017-03-10 17:09:00,2017-03-17 08:37:00,The image shows a streetlight with a broken li...,7,Public Lighting
3,1731179577-4,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.0.jpeg?e7281615,Prishtinës,365,https://ndreqe.com/photo/365.0.full.jpeg?e7281615,"Page protected 04:57, 2 Apr 2017, Last updated...",Reported in Public Lighting category by anonym...,Ferid Curri Street does not have public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00,The image shows a dark street at night with no...,7,Public Lighting
4,1731179577-5,https://ndreqe.com/reports/Komuna+e+Prishtin%C...,"Ndriçim publik 04:57, 2 Pri 2017, ...",https://ndreqe.com/report/365,Raportuar në kategorinë Ndriçim publik nga ano...,*Rruga Ferid Curri nuk ka fare ndriçim publik!,/photo/365.1.jpeg?6898da66,Prishtinës,365,https://ndreqe.com/photo/365.1.jpeg?6898da66,"Public domain 04:57, 2 Apr 2017, Last updated ...",Reported in the Public lighting category by an...,Ferid Curri Street has no public lighting!,2017-04-02 04:57:00,2017-04-03 09:22:00,The image depicts a nighttime scene of a road ...,7,Public Lighting


In [211]:
sample_df.category.value_counts()

,count
category,
Rubbish,52
Public Lighting,25
Pothole in the Road,11
Damaged Sidewalk,7
Other,5


In [212]:
# Save file
sample_df.to_csv('sample_df-4-final.csv', index=False)
print('File saved successfully!')

File saved successfully!
